In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PowerTransformer
from pystan import StanModel
from scipy.interpolate import splev
from fbprophet import Prophet
import pmdarima as pm
import itertools

def RMSE(a, b):
    return mean_squared_error(a,b)**0.5

ERROR:fbprophet.plot:Importing plotly failed. Interactive plots will not work.


In [10]:
a=5
b=6

In [3]:
hs_type1_rmse=[]
hs_type2_rmse=[]
hs_type3_rmse=[]
hs_type4_rmse=[]
hs_type5_rmse=[]
hs_mean_rmse=[]

failure_df = pd.read_csv('failure_count.csv').set_index('age')
engine_df = pd.read_csv('engine.csv').set_index('ship')
test_failure_df = pd.read_csv('failure_count_test.csv').set_index('age')
test_engine_df = pd.read_csv('engine_test.csv').set_index('ship')

for y in range(a, b): # 타입
    ships = engine_df.index[engine_df['engine'] == y]
    for z in range(1, len(ships)): # 배
        temp = failure_df[ships]
        test = failure_df[ships].iloc[:,z-1]
        temp.iloc[:,z-1] = test_failure_df.iloc[:,y-1]

        failure_df[ships] = temp
        test_failure_df.iloc[:,y-1] = test


        # Power Transformer
        pt = PowerTransformer()
        pt.fit(failure_df)
        failure_df = pd.DataFrame(pt.transform(failure_df),
                                  index=failure_df.index, columns=failure_df.columns)
        pt.fit(test_failure_df)
        test_failure_df = pd.DataFrame(pt.transform(test_failure_df), 
                                       index=test_failure_df.index, 
                                       columns=test_failure_df.columns)
        
        
#         # Standard Scaler
#         u = np.nanmean(failure_df.values.flatten())
#         s = np.sqrt(np.nanvar(failure_df.values.flatten()))
#         failure_df = pd.DataFrame((failure_df - u) / s, index=failure_df.index, columns=failure_df.columns)
#         engine_df = pd.read_csv('engine.csv').set_index('ship')
#         test_failure_df = pd.DataFrame((test_failure_df - u) / s, index=test_failure_df.index, columns=test_failure_df.columns)
#         test_engine_df = pd.read_csv('engine_test.csv').set_index('ship')

        # B-spline
        degree = 3
        knots = np.linspace(1, 31, num=5)
        knots_padded = np.concatenate((np.full(degree, 1), knots, np.full(degree, 31)))
        basis = []
        for i in range(knots_padded.shape[0]):
            c = np.zeros(knots_padded.shape[0])
            c[i] = 1
            basis.append(splev(np.arange(1, 32), (knots_padded, c, degree)))
        basis_df = pd.DataFrame(basis).transpose().set_index(np.arange(1, 32))

        
        # Stan code
        model_code ='''
        data {
            int<lower=1> K; // number of knots
            int<lower=1> N; // number of datapoints
            int<lower=1> T; // maximum age
            int<lower=1> S; // number of ships
            int<lower=1> E; // number of engines
            int<lower=1> age[N];
            int<lower=1> ship[N];
            int<lower=1> engine[S];
            matrix[T,K] B;
            vector[N] Y;
        }

        parameters {
            // first layer
            real mu_alpha_bar;
            real<lower=0> sigma_alpha_bar;
            vector[K] mu_w_bar;
            real<lower=0> sigma_w_bar;

            // second layer
            real alpha_bar[E];
            real<lower=0> sigma_alpha;
            vector[K] w_bar[E];
            real<lower=0> sigma_w;

            // third layer
            real alpha[S];
            vector[K] w[S];
            real<lower=0> sigma_y;
        }

        transformed parameters { 
            vector[N] mu;
            for (n in 1:N) {
                mu[n] = alpha[ship[n]] + B[age[n]] * w[ship[n]];
            }
        }

        model {
            mu_alpha_bar ~ normal(0, 1);
            mu_w_bar ~ normal(0, 1);
            sigma_alpha_bar ~ exponential(1);
            sigma_w_bar ~ exponential(1);

            for (e in 1:E) {
                alpha_bar[e] ~ normal(mu_alpha_bar, sigma_alpha_bar);
                w_bar[e] ~ normal(mu_w_bar, sigma_w_bar);
            }
            sigma_alpha ~ gamma(10,10);
            sigma_w ~ gamma(10,10);

            for (s in 1:S) {
                alpha[s] ~ normal(alpha_bar[engine[s]], sigma_alpha);
                w[s] ~ normal(w_bar[engine[s]], sigma_w);
            }
            sigma_y ~ exponential(1);

            Y ~ normal(mu, sigma_y);
        }

        generated quantities{
            vector[N] log_likelihood;
            for (n in 1:N) {
                log_likelihood[n] = normal_lpdf(Y[n]|mu[n], sigma_y);
            }
        }
        '''

        # stan sampling
        sm = StanModel(model_code=model_code)
        K = basis_df.shape[1]
        T = 31
        S = failure_df.shape[1]
        E = np.unique(engine_df['engine']).shape[0]
        Y = failure_df.values[~failure_df.isnull()]
        age_index, ship_index = np.where(~failure_df.isnull())
        data = {
            'K' : K,
            'N' : Y.shape[0],
            'T' : T,
            'S' : S,
            'E' : E,
            'age': age_index + 1,
            'ship': ship_index + 1,
            'engine': engine_df.loc[failure_df.columns, 'engine'],
            'Y': Y,
            'B': basis_df,
        }

        fit = sm.sampling(data=data)
        fit_df = fit.to_dataframe().mean()
        samples = fit.extract()

        # MCMC results
        alpha = np.array([fit_df[f'alpha[{s+1}]'] for s in range(S)])
        w = np.array([[fit_df[f'w[{s+1},{k+1}]'] for k in range(K)] for s in range(S)])
        mu = np.tile(np.expand_dims(alpha, 0), (T, 1)) + basis_df.values @ w.transpose()
        alpha_bar = np.array([fit_df[f'alpha_bar[{e+1}]'] for e in range(E)])
        w_bar = np.array([[fit_df[f'w_bar[{e+1},{k+1}]'] for k in range(K)] for e in range(E)])
        mu_bar = np.tile(np.expand_dims(alpha_bar, 0), (T, 1)) + basis_df.values @ w_bar.transpose()
        mu_alpha_bar = fit_df['mu_alpha_bar']
        mu_w_bar = np.array([fit_df[f'mu_w_bar[{k+1}]'] for k in range(K)])
        mu_zero = np.repeat(mu_alpha_bar, failure_df.shape[0]) + basis_df.values @ mu_w_bar
#         fig, ax = plt.subplots(figsize=(12, 6))
#         ax.set_xlabel('age')
#         ax.set_ylabel('failures')
#         ax.scatter(failure_df.index[age_index], Y, color='k', alpha=0.1)
#         ax.plot(np.arange(1, T + 1), mu, color='g', alpha=0.3, linewidth=0.9)
#         ax.plot(np.arange(1, T + 1), mu_bar, color='m', alpha=1, linewidth=1.5)
#         ax.plot(np.arange(1, T + 1), mu_zero, color='b', alpha=1, linewidth=2)
#         plt.show()


        # hs train type
        print('HS train type')
        y_bar_rmse = []
        for j in range(a, b):
            ships = engine_df.index[engine_df['engine'] == j]
            temp = []
            for i in range(0, failure_df[ships].shape[1]):
                if len(failure_df[ships].iloc[:,i].dropna()) > 3 :
                    comp = pd.concat([pd.DataFrame(mu_bar).iloc[:,j-1], failure_df[ships].iloc[:,i]], axis=1).dropna()
                    y_bar_rmse.append(RMSE(comp.iloc[:,0], comp.iloc[:,1]))
                    temp.append(RMSE(comp.iloc[:,0], comp.iloc[:,1]))
            print(j,  np.mean(temp))
            if j == 1:
                hs_type1_rmse.append(np.mean(temp))
            elif j==2:
                hs_type2_rmse.append(np.mean(temp))
            elif j==3:
                hs_type3_rmse.append(np.mean(temp))
            elif j==4:
                hs_type4_rmse.append(np.mean(temp))
            elif j==5:
                hs_type5_rmse.append(np.mean(temp))
        print(np.mean(y_bar_rmse))
        hs_mean_rmse.append(np.mean(y_bar_rmse))


/Users/choibaroti/opt/anaconda3/envs/stan/lib/python3.7/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/choibaroti/opt/anaconda3/envs/stan/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:2995: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_e7a8868a0e8519c6e59ec95b042c9eb1 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)


HS train type
4 0.9513590508728027
0.9513590508728027


/Users/choibaroti/opt/anaconda3/envs/stan/lib/python3.7/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/choibaroti/opt/anaconda3/envs/stan/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:2995: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_e7a8868a0e8519c6e59ec95b042c9eb1 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)


HS train type
4 0.9631759767702746
0.9631759767702746


/Users/choibaroti/opt/anaconda3/envs/stan/lib/python3.7/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/choibaroti/opt/anaconda3/envs/stan/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:2995: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_e7a8868a0e8519c6e59ec95b042c9eb1 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)


...

HS train type
4 0.9661595362147835
0.9661595362147835


/Users/choibaroti/opt/anaconda3/envs/stan/lib/python3.7/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/choibaroti/opt/anaconda3/envs/stan/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:2995: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_e7a8868a0e8519c6e59ec95b042c9eb1 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)


HS train type
4 0.968558958800899
0.968558958800899


/Users/choibaroti/opt/anaconda3/envs/stan/lib/python3.7/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/choibaroti/opt/anaconda3/envs/stan/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:2995: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_e7a8868a0e8519c6e59ec95b042c9eb1 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)


HS train type
4 0.9651050429031092
0.9651050429031092


In [4]:
print(np.mean(hs_type1_rmse))
print(np.mean(hs_type2_rmse))
print(np.mean(hs_type3_rmse))
print(np.mean(hs_type4_rmse))
print(np.mean(hs_type5_rmse))

nan
nan
nan
0.9657859870002609
nan


/Users/choibaroti/opt/anaconda3/envs/stan/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/choibaroti/opt/anaconda3/envs/stan/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [6]:
# prophet test type

import warnings
warnings.filterwarnings('ignore')

def predict_prophet(s):
    ds = pd.date_range('2020-01-01', periods=s.shape[0], freq='12M')
    training_df = pd.DataFrame({'ds': ds, 'y': s})
    m = Prophet()
    m.fit(training_df)
    predict_df = pd.DataFrame({'ds': ds})
    return m.predict(predict_df)['yhat']

prophet_type1_rmse=[]
prophet_type2_rmse=[]
prophet_type3_rmse=[]
prophet_type4_rmse=[]
prophet_type5_rmse=[]
prophet_mean_rmse=[]

failure_df = pd.read_csv('failure_count.csv').set_index('age')
engine_df = pd.read_csv('engine.csv').set_index('ship')
test_failure_df = pd.read_csv('failure_count_test.csv').set_index('age')
test_engine_df = pd.read_csv('engine_test.csv').set_index('ship')

for y in range(a, b): # 타입
    ships = engine_df.index[engine_df['engine'] == y]
    for z in range(1, len(ships)): # 배
        temp = failure_df[ships]
        test = failure_df[ships].iloc[:,z-1]
        temp.iloc[:,z-1] = test_failure_df.iloc[:,y-1]

        failure_df[ships] = temp
        test_failure_df.iloc[:,y-1] = test


        # Power Transformer
        pt = PowerTransformer()
        pt.fit(failure_df)
        failure_df = pd.DataFrame(pt.transform(failure_df),
                                  index=failure_df.index, columns=failure_df.columns)
        pt.fit(test_failure_df)
        test_failure_df = pd.DataFrame(pt.transform(test_failure_df), 
                                       index=test_failure_df.index, 
                                       columns=test_failure_df.columns)


        test_prophet_rmse =[]
        y_bar_prophet = []
        temp =[]
        print('prophet test type')
        for j in range(a, b):
            ships = engine_df.index[engine_df['engine'] == j]
            for i in range(0, failure_df[ships].shape[1]):
                if len(failure_df[ships].iloc[:,i].dropna()) > 3:
                    y_bar_prophet=(predict_prophet(failure_df[ships].mean(axis=1)))
                    comp = pd.concat([(y_bar_prophet), failure_df[ships].iloc[:,i]], axis=1).dropna()
                    test_prophet_rmse.append(RMSE(comp.iloc[:,0], comp.iloc[:,1]))
                    temp.append(RMSE(comp.iloc[:,0], comp.iloc[:,1]))
            print(j,  np.mean(temp))
            if j == 1:
                prophet_type1_rmse.append(np.mean(temp))
            elif j==2:
                prophet_type2_rmse.append(np.mean(temp))
            elif j==3:
                prophet_type3_rmse.append(np.mean(temp))
            elif j==4:
                prophet_type4_rmse.append(np.mean(temp))
            elif j==5:
                prophet_type5_rmse.append(np.mean(temp))
#         print(np.mean(test_prophet_rmse))
        prophet_mean_rmse.append(np.mean(test_prophet_rmse))


prophet test type


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

4 1.0597915879167972
prophet test type


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

4 1.0580907969124667
prophet test type


...

4 1.058779190799182
prophet test type


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

4 1.0372000380638882
prophet test type


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

4 1.0537162771231432


In [7]:
np.mean(prophet_mean_rmse)
print(np.mean(prophet_type1_rmse))
print(np.mean(prophet_type2_rmse))
print(np.mean(prophet_type3_rmse))
print(np.mean(prophet_type4_rmse))
print(np.mean(prophet_type5_rmse))

nan
nan
nan
1.0594092905103198
nan


In [27]:
# arima test type
import warnings
warnings.filterwarnings('ignore')


def predict_arima(s):
    pn = len(list(itertools.takewhile(np.isnan, s)))
    tn = len(list(itertools.takewhile(np.isnan, s[::-1])))
    if tn:
        m = pm.auto_arima(s[pn:-tn])
        return np.concatenate(
            (np.full(pn, np.nan)),
            m.predict_in_sample(),
            m.predict(tn),
        )
    else:
        m = pm.auto_arima(s[pn:])
        return np.concatenate((
            np.full(pn, np.nan),
            m.predict_in_sample(),
        ))
    
def pred_arima(s):
    m1 = pm.auto_arima(s,error_action='ignore', seasonal=True, m=2)
    y_bar_arima=pd.DataFrame(m1.predict(n_periods=s.shape[0]))
    return y_bar_arima

def pred_arima2(s):
    m1 = pm.auto_arima(s,error_action='ignore', seasonal=True, m=1)
    y_bar_arima=pd.DataFrame(m1.predict(n_periods=s.shape[0]))
    return y_zero_arima

    
def moving_average(a, n = 3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n



arima_type1_rmse=[]
arima_type2_rmse=[]
arima_type3_rmse=[]
arima_type4_rmse=[]
arima_type5_rmse=[]
arima_mean_rmse=[]

failure_df = pd.read_csv('failure_count.csv').set_index('age')
engine_df = pd.read_csv('engine.csv').set_index('ship')
test_failure_df = pd.read_csv('failure_count_test.csv').set_index('age')
test_engine_df = pd.read_csv('engine_test.csv').set_index('ship')

for y in range(a, b): # 타입
    ships = engine_df.index[engine_df['engine'] == y]
    for z in range(0, len(ships)): # 배
        temp = failure_df[ships]
        test = failure_df[ships].iloc[:,z-1]
        temp.iloc[:,z-1] = test_failure_df.iloc[:,y-1]

        failure_df[ships] = temp
        test_failure_df.iloc[:,y-1] = test


        # Power Transformer
        pt = PowerTransformer()
        pt.fit(failure_df)
        failure_df = pd.DataFrame(pt.transform(failure_df),
                                  index=failure_df.index, columns=failure_df.columns)
        pt.fit(test_failure_df)
        test_failure_df = pd.DataFrame(pt.transform(test_failure_df), 
                                       index=test_failure_df.index, 
                                       columns=test_failure_df.columns)


        test_arima_rmse =[]
        y_bar_arima=[]
        print('ARIMA test type')
        for j in range(a, b):
            ships = engine_df.index[engine_df['engine'] == j]
            keys=failure_df[ships].mean(axis=1).dropna().reset_index().age
            temp = []
            if j < 5 :
                y_bar_arima = pred_arima(np.array(failure_df[ships].mean(axis=1).dropna().values))
            else : 
#                 y_bar_arima = pred_arima2(np.array(failure_df[ships].mean(axis=1).dropna().values))
                y_bar_arima = predict_arima(failure_df.mean(axis=1).values)[0:6]
                
            for i in range(0, failure_df[ships].shape[1]):
                if len(failure_df[ships].iloc[:,i].dropna()) > 3:
                    comp = pd.concat([failure_df[ships].iloc[:,i], 
                                      pd.DataFrame(y_bar_arima).set_index(keys)], axis=1).dropna()
                    test_arima_rmse.append(RMSE(comp.iloc[:,0], comp.iloc[:,1]))
                    temp.append(RMSE(comp.iloc[:,0], comp.iloc[:,1]))
                    
            print(j,  np.mean(temp))
            if j == 1:
                arima_type1_rmse.append(np.mean(temp))
            elif j==2:
                arima_type2_rmse.append(np.mean(temp))
            elif j==3:
                arima_type3_rmse.append(np.mean(temp))
            elif j==4:
                arima_type4_rmse.append(np.mean(temp))
            elif j==5:
                arima_type5_rmse.append(np.mean(temp))

        arima_mean_rmse.append(np.mean(test_arima_rmse))


ARIMA test type
5 1.0976085125482475
ARIMA test type
5 1.091230460001305
ARIMA test type
5 1.1111889678514288
ARIMA test type
5 1.0848286727565974


In [28]:
print(np.mean(arima_type1_rmse))
print(np.mean(arima_type2_rmse))
print(np.mean(arima_type3_rmse))
print(np.mean(arima_type4_rmse))
print(np.mean(arima_type5_rmse))

nan
nan
nan
nan
1.0962141532893948


In [48]:
hs_test =[]
for i in range(0, 5):
    test_failure_df.iloc[:,5:11].iloc[:,i]
    comp = pd.concat([pd.DataFrame(mu_zero), test_failure_df.iloc[:,5:11].iloc[:,i]], axis=1).dropna()
    hs_test.append(RMSE(comp.iloc[:,0], comp.iloc[:,1]))
print(hs_test)
print(np.mean(hs_test))

prophet_test=[]
for i in range(0, 5):
    test_failure_df.iloc[:,5:11].iloc[:,i]
    comp = pd.concat([pd.DataFrame(predict_prophet(failure_df.mean(axis=1))), test_failure_df.iloc[:,5:11].iloc[:,i]], axis=1).dropna()
    prophet_test.append(RMSE(comp.iloc[:,0], comp.iloc[:,1]))
print(prophet_test)
print(np.mean(prophet_test))

arima_test=[]
for i in range(0, 5):
    test_failure_df.iloc[:,5:11].iloc[:,i]
    comp = pd.concat([pd.DataFrame(predict_arima(failure_df.mean(axis=1))), test_failure_df.iloc[:,5:11].iloc[:,i]], axis=1).dropna()
    arima_test.append(RMSE(comp.iloc[:,0], comp.iloc[:,1]))
print(arima_test)
print(np.mean(arima_test))

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 23.


[1.034097914635009, 0.920789711001001, 1.1002009080122632, 0.9468712367195368, 0.9841037469608676]
0.9972127034657357


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 23.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 23.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 23.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

[1.0050644030697446, 1.0073174146144346, 1.0161519613730505, 1.010280833380596, 1.001420259315724]
1.00804697435071
[1.0422384819931776, 0.9992219363921243, 1.061051359000441, 1.0354840905811453, 0.9545385925014466]
1.0185068920936668


In [ ]:
comp

In [ ]:
rmse_hs = []
rmse_prophet = []
rmse_arima = []
smape_hs = []
smape_prophet = []
smape_arima = []
for e in range(1, 6):
    test_ships = test_engine_df.index[test_engine_df['engine'] == e]
    test_Y = test_failure_df[test_ships].values[~test_failure_df[test_ships].isnull()]
    test_age_index, test_ship_index = np.where(~test_failure_df[test_ships].isnull())
    
    rmse_hs.append(RMSE(test_Y, mu_bar[test_age_index, e - 1]))
    rmse_prophet.append(RMSE(test_Y, y_bar_prophet[test_age_index, e - 1]))
    rmse_arima.append(RMSE(test_Y, y_bar_arima[test_age_index, e - 1]))
print(rmse_hs)
print(rmse_prophet)
print(rmse_arima)
#     smape_hs.append(SMAPE(test_Y, mu_bar[test_age_index, e - 1]))
#     smape_prophet.append(SMAPE(test_Y, y_bar_prophet[test_age_index, e - 1]))
#     smape_arima.append(SMAPE(test_Y, y_bar_arima[test_age_index, e - 1]))
    

    
    
fig, ax = plt.subplots(figsize=(8, 4))
print(pd.DataFrame({'engine': range(1, 6),
              'rmse_hs': rmse_hs,
              'rmse_prophet': rmse_prophet,
              'rmse_arima': rmse_arima}))

pd.DataFrame({'engine': range(1, 6),
              'rmse_hs': rmse_hs,
              'rmse_prophet': rmse_prophet,
              'rmse_arima': rmse_arima}).set_index('engine').plot.bar(ax=ax)


# fig, ax = plt.subplots(figsize=(12, 4))
# print(pd.DataFrame({'engine': range(1, 6),
#               'smape_hs': smape_hs,
#               'smape_prophet': smape_prophet,
#               'smape_arima': smape_arima}))

# pd.DataFrame({'engine': range(1, 6),
#               'smape_hs': smape_hs,
#               'smape_prophet': smape_prophet,
#               'smape_arima': smape_arima}).set_index('engine').plot.bar(ax=ax)

# fig, ax = plt.subplots(figsize=(12, 6))
# ax.set_xlabel('ship')
# ax.set_ylabel('age')
# ax.matshow(failure_df)
# # plt.show()

In [ ]:
rmse_hs = []
rmse_prophet = []
rmse_arima = []
smape_hs = []
smape_prophet = []
smape_arima = []
for e in range(6, 11):
    test_ships = test_engine_df.index[test_engine_df['engine'] == e]
    test_Y = test_failure_df[test_ships].values[~test_failure_df[test_ships].isnull()]
    test_age_index, test_ship_index = np.where(~test_failure_df[test_ships].isnull())
    rmse_hs.append(RMSE(test_Y, mu_zero[test_age_index]))
    rmse_prophet.append(RMSE(test_Y, y_zero_prophet[test_age_index]))
    rmse_arima.append(RMSE(test_Y, y_zero_arima[test_age_index]))

    
fig, ax = plt.subplots(figsize=(8, 4))
print(pd.DataFrame({'engine': range(6, 11),
              'rmse_hs': rmse_hs,
              'rmse_prophet': rmse_prophet,
              'rmse_arima': rmse_arima}))
pd.DataFrame({'engine': range(6, 11),
              'rmse_hs': rmse_hs,
              'rmse_prophet': rmse_prophet,
              'rmse_arima': rmse_arima}).set_index('engine').plot.bar(ax=ax)


print(np.mean(rmse_hs))
print(np.mean(rmse_prophet))
print(np.mean(rmse_arima))

# fig, ax = plt.subplots(figsize=(12, 4))
# print(pd.DataFrame({'engine': range(6, 11),
#               'smape_hs': smape_hs,
#               'smape_prophet': smape_prophet,
#               'smape_arima': smape_arima}))

# pd.DataFrame({'engine': range(6, 11),
#               'smape_hs': smape_hs,
#               'smape_prophet': smape_prophet,
#               'smape_arima': smape_arima}).set_index('engine').plot.bar(ax=ax)